## Script to bundle all resources in folder as a batch, transaction, collection, or message

- python version 3.6+
- get data from folder
- bundle
- validate and save

In [37]:
from json import load, dumps, loads
from IPython import display as D
from requests import get, post, put
from IPython.display import display, Markdown, HTML
from fhir.resources.bundle import Bundle as B, BundleEntry as BE, BundleEntryRequest as BER, BundleEntryResponse as BERes, BundleEntrySearch as BES 
import os, uuid
from datetime import datetime
from pprint import pprint

# bundle_id = 'cdex-data-requirements-bundle-example1'
bundle_id = 'cdex-scenario7-payer-load-transaction-bundle'
bundle_type = 'transaction'


###  write to file

In [38]:
def write_file(r): # write file
    #out_path  '/Users/ehaas/Documents/FHIR/Davinci-DEQM/source/examples/'  # append forward slash
    out_path = in_path
    with open(f'{out_path}/Bundle-{bundle_id}.json', 'w') as f:
        f.write(r)


### open json file and return as dict

In [39]:
def open_file(f_name): # get files
    with open(f'{in_path}/{f_name}') as f:
        r = f.read()
        return(loads(r))

### Get files in current path

In [40]:
in_path = 'r4/CDex_PAS_Payer_Load'

files = [x for x in os.listdir(in_path) if x.endswith(".json") and not x.startswith("Bundle")]
pprint(files)


['example-1-patient.json', 'cdex-claim-MRIAuthorizationBundleRIExample.json']


### create Bundle 'b'  change the id for unique Bundles!!!

In [41]:
b = B(
    id = bundle_id,
    type = bundle_type,
    timestamp = datetime.now().astimezone().isoformat(),
)
b.json()

'{"resourceType":"Bundle","id":"cdex-scenario7-payer-load-transaction-bundle","type":"transaction","timestamp":"2023-08-23T23:55:01.066830-07:00"}'

### Add resources to bundle

#### Create a mapping of ids to uuids

- get id from resource
- map to uuid (create a dict of {urn, (old_ref)})
- remove id element
- replace all old refs in bundle with new urns


In [42]:
b.entry = []
ref_map = {}
for i in files:
    r = open_file(i)
    # remove text adn id elements from Bundle
    r.pop('text', None) # remove id elements from Bundle
    old_id = r.pop('id',None) # remove text elements from Bundle
    try:
        print(r['url'])
        new_urn = r['url']
    except:
        new_urn = uuid.uuid1().urn # new urn for resource
    ref_map[new_urn] = f'{r["resourceType"]}/{old_id}'
    # print( id_map , '\n')
    e = dict( 
        fullUrl = new_urn,
        resource = r,
      )
    print(e)
    if bundle_type in ['transaction', 'batch']:
        e['request'] = dict(
                    method = 'POST',
                    url = r['resourceType']
                    )
                    
    
    b.entry.append(e)
b_json = b.json()
for new_ref, old_ref in ref_map.items():
    b_json = b_json.replace(old_ref,new_ref)
print(b_json)
write_file(b_json)


{'fullUrl': 'urn:uuid:254c080e-424b-11ee-a1fc-daaf18ab2cba', 'resource': {'resourceType': 'Patient', 'meta': {'profile': ['http://hl7.org/fhir/us/davinci-cdex/StructureDefinition/cdex-patient-demographics']}, 'identifier': [{'use': 'usual', 'type': {'coding': [{'system': 'http://hl7.org/fhir/us/davinci-hrex/CodeSystem/hrex-temp', 'code': 'UMB', 'display': 'Member Number'}], 'text': 'Member Number'}, 'system': 'http://example.org/cdex/payer/member-ids', 'value': 'Member123'}], 'name': [{'family': 'Shaw', 'given': ['Amy']}], 'birthDate': '1987-02-20'}}
{'fullUrl': 'urn:uuid:254c19e8-424b-11ee-a1fc-daaf18ab2cba', 'resource': {'resourceType': 'Claim', 'identifier': [{'system': 'http://example.org/payer', 'value': 'trackingid123'}, {'system': 'http://CedarArmsMedicalCenter.com/claim', 'value': 'MED-Claim123'}], 'status': 'active', 'type': {'coding': [{'system': 'http://terminology.hl7.org/CodeSystem/claim-type', 'code': 'professional', 'display': 'Professional'}]}, 'use': 'preauthorization'